[<img src="../../header.svg">](../index.ipynb)

---
# Lösen der Poisson-Gleichung mit FEM in UG4 

Wir lösen in diesem Beispiel 
$$-\triangle u = f $$
auf dem Einheitsquadrat mit Dirichlet-Randwerten. 

Dazu verwenden wir hier einen entprechenden Jupyter-Kernel für die Software UG4, der LUA unterstützt. Eine Erweiterung folgt im [nachfolgenden Beispiel](./tutorial-fem-02.ipynb).

## Initialisierung 

In [58]:
import sys
sys.path.append("../skin/")
import modsimtools as util


## Problemdefinition

Zur Problemdefinition definieren wir einige Variablen:

In [59]:
myGridName= "grids/unit_square_tri.ugx" # grids/unit_square_tri.ugx",
myNumRefs= 3
mySubsets = {"Inner", "Boundary"}

Randbedingungen setzen wir über einen Callback:

In [60]:

# Callback fuer Randbedingungen
def myDirichletBndCallback(x, y, t) :
    if (y==1) :
        return true, 0.0 
    elif (y==0) :  
        return true, math.sin(math.pi*1*x)
    else : 
        return false, 0.0 
    

## Rechengebiet

Einlesen aus einer Datei ganz bequem in einer Zeile:

In [61]:
dom = util.CreateDomain(myGridName, myNumRefs, mySubsets)

Loading Domain {gridName}...
Domain loaded.
Refining ...
Refining step {i} ...
Refining step {i} ...
Refining step {i} ...
Refining done


## Ansatzraum
Erzeuge Ansatzraum mit stückweise linearen Funktionen (elementweise Polynome 1. Grades):

In [62]:
approxSpaceDesc = dict(fct = "u", type = "Lagrange", order = 1)
approxSpace = util.CreateApproximationSpace(dom, approxSpaceDesc)

Approximation space:
| ---------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                  |
|  Algebra: Block 1 (divide by 1 for #Index)                                   |
|                                                                              |
|    GridLevel   |       Domain |     0: Inner |  1: Boundary                  |
| ---------------------------------------------------------------------------- |
| (lev,    0)    |            9 |            1 |            8 |
| (lev,    1)    |           25 |            9 |           16 |
| (lev,    2)    |           81 |           49 |           32 |
| (lev,    3)    |          289 |          225 |           64 |
| (lev,    0, g) |            9 |            1 |            8 |
| (lev,    1, g) |           25 |            9 |           16 |
| (lev,    2, g) |           81 |           49 |           32 |
| (lev,    3, g) |          289 |          22

## Diskretisierung

Erzeuge Objekt für eine **Elementdiskretisierung** für die Konvektions-Diffusionsgleichung.

In [70]:
elemDisc = cd.ConvectionDiffusionFE2d("u", "Inner")
elemDisc.set_diffusion(1.0)

# Optional: Setze rechte Seite f
elemDisc.set_source(1.0)

Erzeuge Objekt für **Randbedingungen**:

In [72]:
dirichletBND = ug4.DirichletBoundary2dCPU1()
# dirichletBND.add(myDirichletBndCallback, "u", "Boundary")
dirichletBND.add(0.0, "u", "Boundary")

Füge beides zu einer **Gebietsdiskretisierung** hinzu:

In [73]:
domainDisc = ug4.DomainDiscretization2dCPU1(approxSpace)
domainDisc.add(elemDisc)
domainDisc.add(dirichletBND)

## Konfiguration eines iterativen Lösers

Ein sog. Mehrgitterverfahren (=> Modellierung und Simulation 2) hat lediglich lineare Komplexität

In [74]:
# MG solver setup (using 'util/solver_util.lua')
solverDesc = {
    type = "bicgstab",
    precond = {
        type = "gmg",
        approxSpace = approxSpace,
        smoother = "sgs",
        baseSolver = "lu"
    }
}
solver = util.solver.CreateSolver(solverDesc)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (58224527.py, line 3)

In [75]:
import pysuperlu as slu
solver=slu.SuperLUCPU1()

## Assembliere und löse LGS

In [76]:
A = ug4.AssembledLinearOperatorCPU1(domainDisc)
x = ug4.GridFunction2dCPU1(approxSpace)
b = ug4.GridFunction2dCPU1(approxSpace)

# x.clear(0.0)

domainDisc.assemble_linear(A, b)
domainDisc.adjust_solution(x)

solver.init(A, x)
solver.apply(x, b)

True

### Ausgabe als *.vtu bzw. *.vec-Datei

Wir visualisieren Daten mit pyvista bzw. dem ConnectionViewer:

In [77]:
import pyvista
pyvista.start_xvfb()
pyvista.set_jupyter_backend('trame')

a) Für die Lösung

In [78]:
solFileName = "fem01_solution_u"
ug4.WriteGridFunctionToVTK(x, solFileName)
ug4.SaveVectorForConnectionViewer(x, solFileName + ".vec")

result = pyvista.read("fem01_solution_u.vtu")
result.plot(scalars="u", show_edges=True, cmap='jet')

Widget(value="<iframe src='http://localhost:56144/index.html?ui=P_0x1651a32d0_3&reconnect=auto' style='width: …

b) Für die rechte Seite $b$ und Matrix $A$

In [79]:
solFileName = "fem01_rhs_b"
ug4.WriteGridFunctionToVTK(b, solFileName)
ug4.SaveVectorForConnectionViewer(b, solFileName + ".vec")
ug4.SaveMatrixForConnectionViewer(x, A, "fem01_matrix_A.mat")

In [80]:
result = pyvista.read("fem01_rhs_b.vtu")
result.plot(scalars="u", show_edges=True, cmap='jet')

Widget(value="<iframe src='http://localhost:56144/index.html?ui=P_0x2ad659d90_4&reconnect=auto' style='width: …